# Generate full emissions portfolio from projected Kyoto gas baskets
Updated to include announcements as of COP26, 5/11/21. Including an analysis of full impact of Net Zero commitments on emissions.

NOTE: Most of this file is unnecessary - only the file sr15_cleaned.csv is needed from this!

# Part 1: import and clean data

In [1]:
import sys
stdout = sys.stdout

In [2]:
import country_converter
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import pyam
import re

<IPython.core.display.Javascript object>

In [3]:
from silicone.time_projectors import ExtendLatestTimeQuantile
import silicone.multiple_infillers as mi
import silicone.database_crunchers as dc
from silicone.utils import download_or_load_sr15, convert_units_to_MtCO2_equiv, _construct_consistent_values

In [4]:
version_no = "r10_1"

# Import the data
Data is imported and put into pyam format. Names of regions are unified. Units of measurements are defined and cleaned. 

In [6]:
variable = "Emissions|Kyoto Gases (AR6-GWP100)"
metricname = "AR6GWP100"
kyexaf = variable + "|Excluding CO2 AFOLU"
unit = "Mt CO2-equiv/yr"
co2unit = "Mt CO2/yr"

In [7]:
metadata = pd.read_excel(
    "../input/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name="meta"
).rename(columns={"Model": "model", "Scenario": "scenario"}).set_index(["model", "scenario"])

In [8]:
# Read in and prepare the database

ar6_file_regional = "../input/AR6_Scenarios_Database_R10_regions_v1.1_emissions_price_filtered.csv"
if not os.path.isfile(ar6_file_regional):
    print("You need to download the file from IIASA, this may not work")
    conn = pyam.iiasa.Connection("ar6-public")
    ar6regional = conn.query(timeslice=None, variable=["Emissions|*", "Price|Carbon"], region="R10*")
    ar6regional.to_csv(ar6_file_regional)
    ar6regional = pyam.IamDataFrame(ar6regional, meta=metadata)
else:
    ar6regional = pyam.IamDataFrame(ar6_file_regional, meta=metadata)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ..\input\AR6_Scenarios_Database_R10_regions_v1.1_emissions_price_filtered.csv


In [9]:
ar6regional.meta

Category                    Category_name  \
model           scenario                                                       
AIM/CGE 2.2     EN_INDCi2030_1000f       C3  C3: limit warming to 2°C (>67%)   
                EN_INDCi2030_1200        C4  C4: limit warming to 2°C (>50%)   
                EN_INDCi2030_1200f       C4  C4: limit warming to 2°C (>50%)   
                EN_INDCi2030_1400        C4  C4: limit warming to 2°C (>50%)   
                EN_INDCi2030_1400f       C4  C4: limit warming to 2°C (>50%)   
...                                     ...                              ...   
TIMES-China 2.0 NPi_2020_400            NaN                              NaN   
                PEAK25-H                NaN                              NaN   
                PEAK25-L                NaN                              NaN   
                PEAK30-H                NaN                              NaN   
                PEAK30-L                NaN                              NaN   

                                   Category_subset  \
model           scenario                             
AIM/CGE 2.2     EN_INDCi2030_1000f     C3y_+veGHGs   
                EN_INDCi2030_1200               C4   
                EN_INDCi2030_1200f              C4   
                EN_INDCi2030_1400               C4   
                EN_INDCi2030_1400f              C4   
...                                            ...   
TIMES-China 2.0 NPi_2020_400                   NaN   
                PEAK25-H                       NaN   
                PEAK25-L                       NaN   
                PEAK30-H                       NaN   
                PEAK30-L                       NaN   

                                                                           Subset_Ch4  \
model           scenario                                                                
AIM/CGE 2.2     EN_INDCi2030_1000f  Limit 2C (>67%) or return to 1.5C (>50%) after...   
                EN_INDCi2030_1200                                                 NaN   
                EN_INDCi2030_1200f                                                NaN   
                EN_INDCi2030_1400                                                 NaN   
                EN_INDCi2030_1400f                                                NaN   
...                                                                               ...   
TIMES-China 2.0 NPi_2020_400                                                      NaN   
                PEAK25-H                                                          NaN   
                PEAK25-L                                                          NaN   
                PEAK30-H                                                          NaN   
                PEAK30-L                                                          NaN   

                                   Category_Vetting_historical IMP_marker  \
model           scenario                                                    
AIM/CGE 2.2     EN_INDCi2030_1000f                          C3    non-IMP   
                EN_INDCi2030_1200                           C4    non-IMP   
                EN_INDCi2030_1200f                          C4    non-IMP   
                EN_INDCi2030_1400                           C4    non-IMP   
                EN_INDCi2030_1400f                          C4    non-IMP   
...                                                        ...        ...   
TIMES-China 2.0 NPi_2020_400                               NaN        NaN   
                PEAK25-H                                   NaN        NaN   
                PEAK25-L                                   NaN        NaN   
                PEAK30-H                                   NaN        NaN   
                PEAK30-L                                   NaN        NaN   

                                                 Literature Reference (if applicable)  \
model           scenario                  

In [10]:
ar6file = "../input/AR6_Scenarios_Database_World_v1.1.csv"
if not os.path.isfile(ar6file):
    print("You need to download the file from IIASA, this may not work")
    conn = pyam.iiasa.Connection("ar6-public")
    ar6 = conn.query(timeslice=None, varible=["Emissions|*", "Price|Carbon"], region="World*")
    ar6.to_csv(ar6file)
else:
    ar6 = pyam.IamDataFrame(ar6file, meta=metadata)

pyam.core - INFO: Reading file ..\input\AR6_Scenarios_Database_World_v1.1.csv


In [11]:
metadata["Category_Vetting_historical"].unique()

array(['failed_Vetting_historical_failed-vetting', 'C3', 'C5', 'C6', 'C7',
       'C4', 'C1', 'no-climate-assessment', 'C2', 'C8', 'failed-vetting'],
      dtype=object)

In [12]:
ar6 = ar6.filter(Category_Vetting_historical=["failed_Vetting_historical_failed-vetting", "failed-vetting"], keep=False)

In [13]:
ar6regional = ar6regional.filter(
    Category_Vetting_historical=["failed_Vetting_historical_failed-vetting", "failed-vetting"], keep=False
)

In [14]:
# The metadata is in conflict 
ar6 = pyam.concat([ar6.filter(region="World"), pyam.IamDataFrame(ar6regional.timeseries())])

In [15]:
ar6.region

['R10AFRICA',
 'R10CHINA+',
 'R10EUROPE',
 'R10INDIA+',
 'R10LATIN_AM',
 'R10MIDDLE_EAST',
 'R10NORTH_AM',
 'R10PAC_OECD',
 'R10REF_ECON',
 'R10REST_ASIA',
 'R10ROWO',
 'World']

In [16]:
co2 = "Emissions|CO2"
co2ind = "Emissions|CO2|Energy and Industrial Processes"
co2afolu = "Emissions|CO2|AFOLU"
kyotos = [co2ind, co2afolu, "Emissions|CH4", "Emissions|N2O", "Emissions|F-Gases"]

In [17]:
# Correct a difficult unit
ar6 = ar6.rename({"unit":{"kt HFC43-10/yr": "kt HFC4310mee/yr"}})

In [18]:
ar6 = ar6.filter(variable=["Emissions*", "Price|Carbon"]).filter(level="3+", keep=False)

In [19]:
unit_lookup = ar6.data[["variable", "unit"]].drop_duplicates().set_index("variable")

In [20]:
# Import historical database
histall = pd.read_excel("../input/VanMarle2017Totals.xlsx", skiprows=1)
del histall["Unnamed: 13"]
del histall["Unnamed: 14"]
del histall["Unnamed: 15"]
histall = histall.rename(columns={"Scenario (History)": "Scenario"})
histall["Variable"] = histall["Variable"].str.replace("CMIP6 ", "")
histall["Scenario"] = "History"
histall["Region"] = histall["Region"].str.replace(".2", "").replace("R5OECD", "R5OECD90+EU")
histall = pyam.IamDataFrame(histall)

C:\Users\rlamboll\Anaconda3\envs\silicone-1\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


In [21]:
kyoto_gases = [
        'Emissions|C2F6', 
        'Emissions|CF4',
        'Emissions|CH4',
        'Emissions|CO2',
        'Emissions|HFC',
        'Emissions|SF6',
        'Emissions|N2O'
]
fgases = ["Emissions|HFC", 'Emissions|PFC|C2F6', 'EmissionsPFC|CF4', 'Emissions|SF6']
pfcs = ['Emissions|PFC|C2F6', 'Emissions|PFC|CF4']
histkyoto = _construct_consistent_values(
    variable, 
    kyoto_gases, 
    convert_units_to_MtCO2_equiv(histall.filter(region="World", variable=kyoto_gases), metric_name=metricname)
)
histall = histall.rename(variable={"Emissions|CF4":"Emissions|PFC|CF4", 'Emissions|C2F6':'Emissions|PFC|C2F6'})
histkyoto.timeseries()
histall = histall.append(histkyoto)
histnonafolu = histall.subtract(
    variable,
    co2afolu,
    name=kyexaf,
    ignore_units=True
)
histnonafolu = histnonafolu.data
histnonafolu["unit"] = unit
histnonafolu = pyam.IamDataFrame(histnonafolu)
histco2eni = histall.subtract(
    co2,
    co2afolu,
    name=co2ind,
    ignore_units=True
).data
histco2eni["unit"] = "Mt CO2/yr"
histco2eni = pyam.IamDataFrame(histco2eni)
histfgas = _construct_consistent_values(
    "Emissions|F-Gases", 
    fgases, 
    convert_units_to_MtCO2_equiv(histall.filter(region="World", variable=fgases), metric_name=metricname)
)
histpfc = _construct_consistent_values(
    "Emissions|PFC", 
    pfcs, 
    convert_units_to_MtCO2_equiv(histall.filter(region="World", variable=pfcs), metric_name=metricname)
)
histall = histall.append(histnonafolu).append(histco2eni).append(histfgas).append(histpfc)

In [22]:
histall.timeseries()

1990  \
model   scenario region variable           unit                      
History History  R5ASIA Emissions|BC       Mt BC/yr       2.943000   
                        Emissions|CH4      Mt CH4/yr     92.858640   
                        Emissions|CO       Mt CO/yr     299.179721   
                        Emissions|CO2      Mt CO2/yr   4089.048733   
                        Emissions|NH3      Mt NH3/yr     23.803027   
...                                                            ...   
                 World  Emissions|PFC|C2F6 kt C2F6/yr     2.110000   
                        Emissions|PFC|CF4  kt CF4/yr     14.810000   
                        Emissions|SF6      kt SF6/yr      4.880000   
                        Emissions|Sulfur   Mt SO2/yr    134.154791   
                        Emissions|VOC      Mt VOC/yr    208.103495   

                                                              1995  \
model   scenario region variable           unit                      
History History  R5ASIA Emissions|BC       Mt BC/yr       3.213000   
                        Emissions|CH4      Mt CH4/yr     99.653799   
                        Emissions|CO       Mt CO/yr     299.379993   
                        Emissions|CO2      Mt CO2/yr   5507.514650   
                        Emissions|NH3      Mt NH3/yr     25.299512   
...                                                            ...   
                 World  Emissions|PFC|C2F6 kt C2F6/yr     2.200000   
                        Emissions|PFC|CF4  kt CF4/yr     10.680000   
                        Emissions|SF6      kt SF6/yr      5.630000   
                        Emissions|Sulfur   Mt SO2/yr    121.397386   
                        Emissions|VOC      Mt VOC/yr    216.248905   

                                                              2000  \
model   scenario region variable           unit                      
History History  R5ASIA Emissions|BC       Mt BC/yr       3.177000   
                        Emissions|CH4      Mt CH4/yr    104.168651   
                        Emissions|CO       Mt CO/yr     308.268244   
                        Emissions|CO2      Mt CO2/yr   6200.737158   
                        Emissions|NH3      Mt NH3/yr     26.153645   
...                                                            ...   
                 World  Emissions|PFC|C2F6 kt C2F6/yr     2.850000   
                        Emissions|PFC|CF4  kt CF4/yr      9.820000   
                        Emissions|SF6      kt SF6/yr      4.860000   
                        Emissions|Sulfur   Mt SO2/yr    111.148729   
                        Emissions|VOC      Mt VOC/yr    195.329257   

                                                              2005  \
model   scenario region variable           unit                      
History History  R5ASIA Emissions|BC       Mt BC/yr       3.910000   
                        Emissions|CH4      Mt CH4/yr    124.903493   
                        Emissions|CO       Mt CO/yr     354.636749   
                        Emissions|CO2      Mt CO2/yr   9237.420318   
                        Emissions|NH3      Mt NH3/yr     28.538322   
...                                                            ...   
                 World  Emissions|PFC|C2F6 kt C2F6/yr     2.260000   
                        Emissions|PFC|CF4  kt CF4/yr     10.390000   
                        Emissions|SF6      kt SF6/yr      5.790000   
                        Emissions|Sulfur   Mt SO2/yr    124.980692   
                        Emissions|VOC      Mt VOC/yr    218.788737   

                                                               2010  \
model   scenario region variable           unit                       
History History  R5ASIA Emissions|BC       Mt BC/yr        4.529000   
                        Emissions|CH4      Mt CH4/yr     145.036835   
                        Emissions|CO       Mt CO/yr      383.928799   
                        Emissions|CO2      Mt CO2/yr   12471.21

# Save relevant values


In [23]:
out_folder = f"../output/processed_files/{version_no}/"
os.makedirs(out_folder, exist_ok=True)
ar6.to_csv(out_folder + "ar6_cleaned.csv")
histall.to_csv(out_folder + "histall_cleaned.csv")